In [1]:
# Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 

import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [2]:
# import libraries and connect to the SQLite database.

import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("C:\\Users\\restr\\Documents\\Springboard\\dsc\\Unit 8 - SQL & Databases\\8.3 SQL Case Study - Country Club\\SQLFiles Tier 1\\sqlite_db_pythonsqlite.db")

# Function to execute query and return results in a pandas DataFrame
def execute_query(query):
    return pd.read_sql_query(query, conn)


In [3]:
#Use the CASE statement within the SUM function to differentiate between member bookings (memid not equal to 0) 
# and guest bookings (memid equals 0). For member bookings, 
# we multiply the membercost with slots; for guest bookings, we multiply the guestcost with slots.
# Use HAVING clause to filter out facilities that made a revenue of 1000 or more.

query10 = """
SELECT name, 
       SUM(CASE WHEN memid = 0 THEN guestcost * slots 
                ELSE membercost * slots END) AS revenue
FROM Bookings
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
GROUP BY name
HAVING revenue < 1000
ORDER BY revenue;
"""
df10 = execute_query(query10)
print(df10)


            name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


In [4]:
# Join the Members table with itself. The join is on recommendedby and memid.
#Select the full name of the member (m1.surname || ', ' || m1.firstname) and the full name of the member who recommended 
#them (m2.surname || ', ' || m2.firstname).
# Use the WHERE clause to exclude members who were not recommended by anyone.

query11 = """
SELECT m1.surname || ', ' || m1.firstname AS member, 
       m2.surname || ', ' || m2.firstname AS recommended_by
FROM Members m1
INNER JOIN Members m2 ON m1.recommendedby = m2.memid
WHERE m1.recommendedby IS NOT NULL AND m1.recommendedby != ''
ORDER BY member;
"""
df11 = execute_query(query11)
print(df11)


                      member      recommended_by
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8           Genting, Matthew     Butters, Gerald
9                 Hunt, John  Purview, Millicent
10              Jones, David    Joplette, Janice
11            Jones, Douglas        Jones, David
12          Joplette, Janice       Smith, Darren
13           Mackenzie, Anna       Smith, Darren
14             Owen, Charles       Smith, Darren
15             Pinker, David     Farrell, Jemima
16        Purview, Millicent        Smith, Tracy
17         Rumney, Henrietta    Genting, Matthew
18         Sarwin, Ramnaresh     Bader, Florence
19               Smi

In [5]:
# Join the Bookings table with the Facilities table based on facid.
# Use the WHERE clause to exclude guest bookings (memid equals 0).
# Group by name of the facility and count the number of bookings for each.

query12 = """
SELECT name, COUNT(*) AS member_usage
FROM Bookings
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
WHERE memid != 0
GROUP BY name;
"""
df12 = execute_query(query12)
print(df12)


              name  member_usage
0  Badminton Court           344
1   Massage Room 1           421
2   Massage Room 2            27
3       Pool Table           783
4    Snooker Table           421
5     Squash Court           195
6     Table Tennis           385
7   Tennis Court 1           308
8   Tennis Court 2           276


In [6]:
# Join the Bookings table with the Facilities table based on facid.
# Use the WHERE clause to exclude guest bookings (memid equals 0).
# Group by name of the facility and count the number of bookings for each.

query13 = """
SELECT name, strftime('%m', starttime) AS month, COUNT(*) AS usage
FROM Bookings
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
WHERE memid != 0
GROUP BY name, month;
"""
df13 = execute_query(query13)
print(df13)


               name month  usage
0   Badminton Court    07     51
1   Badminton Court    08    132
2   Badminton Court    09    161
3    Massage Room 1    07     77
4    Massage Room 1    08    153
5    Massage Room 1    09    191
6    Massage Room 2    07      4
7    Massage Room 2    08      9
8    Massage Room 2    09     14
9        Pool Table    07    103
10       Pool Table    08    272
11       Pool Table    09    408
12    Snooker Table    07     68
13    Snooker Table    08    154
14    Snooker Table    09    199
15     Squash Court    07     23
16     Squash Court    08     85
17     Squash Court    09     87
18     Table Tennis    07     48
19     Table Tennis    08    143
20     Table Tennis    09    194
21   Tennis Court 1    07     65
22   Tennis Court 1    08    111
23   Tennis Court 1    09    132
24   Tennis Court 2    07     41
25   Tennis Court 2    08    109
26   Tennis Court 2    09    126


In [8]:
# Close connection

conn.close()